# Python Notebook training NPI on political views

This notebook is also a good example on how to use the NPI project to train your own NPI

## Setup

Prerequisites:

* Follow the instructions in the readme to install the NPI repository before running this notebook.
* Clone the [NewB repo](https://github.com/JerryWei03/NewB) in the data/external folder.

In [66]:
# For dev purposes, enable autoreload of modules
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import pandas as pd
import torch

%autoreload

# Constructing training dataset
from npi.dataset import NPIDatasetConstructor, NPIDatasetLoader

# Models
from npi.models import NPITrainingModels

# Training NPI
from npi.training import NPIStyleTrainer, NPITrainer

# Configuration
from npi.config import NPIConfig



Notice that each step in training an NPI is conditioned on the variable toggles, so you can disable certain steps to not redo them again when running the whole notebook.

In [83]:
# Toggle steps
construct = True
train_class = True
test_class = True
train_npi = True


In [69]:
model_layers = [5, 11]

device = torch.device(f"cuda:0")
config = NPIConfig(
    device, gpt_model="gpt2", perturbation_indices=model_layers, npi_name="politics"
)


## Create Dataset

### Get Data from NewB GitHub repo

See `construct_politics_data` to see how to construct data.

In [87]:
if construct:
    with open("../../data/external/NewB/train_orig.txt", "r", newline="") as f:
        data = pd.read_csv(f, delimiter="\t", names=["view", "text"])
        data["view"] = data["view"].replace(
            to_replace={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}
        )
        data = data[data.view != 5]
        data = data.sample(frac=1)
        data_iter = zip(data["text"], data["view"])
        construct_data = NPIDatasetConstructor(config)
        construct_data.construct_dataset(data_iter, len(data))


Using cuda:0
gpu_device is cuda:0


  4%|▍         | 8706/230710 [11:43<5:44:35, 10.74it/s]

## Train classifier from Dataset



In [86]:
model_loader = NPITrainingModels(config)
dataset_loader = NPIDatasetLoader(config, target="style")
style_trainer = NPIStyleTrainer(config, class_lr=1e-5)
classifier = None
if train_class:
    classifier = style_trainer.train_classifier(
        model_loader,
        dataset_loader,
        num_epochs=10,
        # continue_epoch=4,  # Set epoch of classifier to load to continue training.
    )


Classifier INIT
Defining classifier model
Loading style_model weights from models/npi_models/politics_style_model_008.pth
Initializing class loss
Training


epoch: 0 train_loss=69986576.00 test_loss=72479288.00 test_accuracy=0.00:  33%|███▎      | 1/3 [00:00<00:00,  9.93it/s]

Saving Style Classifier epoch 0


epoch: 5 train_loss=69986656.00 test_loss=72480176.00 test_accuracy=0.00:  33%|███▎      | 1/3 [00:00<00:00,  9.71it/s]

Saving Style Classifier epoch 5


Saving Style Classifier epoch 9
Epoch train loss history ==  [69155562.66666667, 69155525.33333333, 69155517.33333333, 69155509.33333333, 69155496.0, 69155485.33333333, 69155477.33333333, 69155466.66666667, 69155456.0, 69155448.0]


### Test Classifier


In [79]:
_, test_loader, _, _ = dataset_loader.load_train_and_test_dataloaders()

classifier = model_loader.load_style_classifier() if not classifier else classifier
# When output=True, it will log the following in models/npi_models/style_classifier_output.log:
#   (generated text from the dataset)
#   truth={target_label} actual={model_output}
style_trainer.test_model(test_loader, classifier, torch.nn.BCELoss(), output=True)


Classifier INIT
Defining classifier model
Loading style_model weights from models/npi_models/politics_style_model_008.pth


(69738789.6, 0.44000000000000006)

## Training NPI

In [71]:
# Initialize models for training and testing
models = NPITrainingModels(
    config,
)

In [72]:
num_epochs = 6
split_ratio = 0.25
batch_size = 5
headstart = 0  # set at 0 for debugging

if train_npi:
    trainer = NPITrainer(config, batch_size=batch_size, headstart=headstart)
    dataset_loader = NPIDatasetLoader(config, split_ratio=split_ratio)
    trainer.train_adversarial_npi(models, num_epochs, dataset_loader)


## Test NPI

In [73]:
text = "Trump today was found"
print(F"Original GPT2 output: '{text}'\n{models.gpt2_generate_text(text)}")


Initializing GPT2WithNPI model with tokenizer -- not being placed on GPU until npi loss evaluation



Original GPT2 output: 'Trump today was found'
 to have been a liar.

The FBI has been investigating the Trump campaign's ties to Russia.

The Trump campaign has also been accused of using the word "radical" in a tweet.

"I am not a fan of the idea of a "big-screen" movie.

"I'm not going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say


In [74]:
print(F"NPI GPT2 output: '{text}'\n{models.npi_generate_text(text)}")

NPI INITIALIZATION
Loading npi_model weights from models/npi_models/politics_npi_model_000.pth


NPI GPT2 output: 'Trump today was found'
 to have been a liar.

The FBI has been investigating the Trump campaign's ties to Russia.

The Trump campaign has also been accused of using the word "radical" in a tweet.

"I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am
